<h1>Applied Data Science Capstone</h1>

This is a Jupyter Notebook in Python created mainly for the Applied Data Science Capstone Project

In [5]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from geopy.geocoders import ArcGIS
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
import json
import geocoder
import pickle

## Scraping the web link and inserting data into dataframe using Beautifulsoup package

### Business Problem

We are trying to determine, which neighborhood in the city of Pune, India would be ideal for opening a multiplex theatre.

In [1]:
print(' Hello Capstone Project Course!')

 Hello Capstone Project Course!


We will get the list of neighborhoods in Pune city from a wikipedia page.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Pune'## page url
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup ## soup now has the page html

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighbourhoods in Pune - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xmpm5wpAMM8AA@d5YSoAAACI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Pune","wgTitle":"List of neighbourhoods in Pune","wgCurRevisionId":945156032,"wgRevisionId":945156032,"wgArticleId":10345898,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from July 2019","

In [4]:
## scrape neighborhood list from page html stored in variable soup
list = []
for ultag in soup.find_all('ul'):
    for litag in ultag.find_all('li'):
        list.append(litag.text)

In [5]:
list = list[7:]

In [6]:
list = list[0:47]

In [7]:
list

['Ambegaon',
 'Aundh',
 'Baner',
 'Bavdhan Khurd',
 'Bavdhan Budruk',
 'Balewadi',
 'Bhamburde (now called Shivajinagar)',
 'Bibvewadi',
 'Bhugaon',
 'Bhukum',
 'Dhankawadi',
 'Dhanori',
 'Dhayari',
 'Erandwane',
 'Fursungi',
 'Ghorpadi',
 'Hadapsar',
 'Hingne Khurd',
 'Hingne Budruk (now Karvenagar)',
 'Kalas',
 'Katraj',
 'Khadki',
 'Kharadi',
 'Kondhwa',
 'Koregaon Park',
 'Kothrud',
 'Manjri',
 'Markal',
 'Mohammedwadi',
 'Mundhwa',
 'Nanded',
 'Parvati',
 'Panmala',
 'Pashan',
 'Pirangut',
 'Shivane',
 'Sus',
 'Undri',
 'Vishrantwadi',
 'Vitthalwadi',
 'Vadgaon Khurd',
 'Vadgaon Budruk',
 'Vadgaon Sheri',
 'Wagholi',
 'Wanwadi',
 'Warje',
 'Yerwada']

In [8]:
df = pd.DataFrame(list, columns=['Neighborhood'])

In [9]:
df.head()

,Neighborhood
0,Ambegaon
1,Aundh
2,Baner
3,Bavdhan Khurd
4,Bavdhan Budruk


In [35]:
#dataframe of Pune neighborhoods

df

,Neighborhood
0,Ambegaon
1,Aundh
2,Baner
3,Bavdhan Khurd
4,Bavdhan Budruk
5,Balewadi
6,Shivajinagar
7,Bibvewadi
8,Bhugaon
9,Bhukum


In [48]:
longitude2

[73.8190432]

In [47]:
CLIENT_ID = '****' # your Foursquare ID
CLIENT_SECRET = '****

In [11]:
with open('df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [52]:
## getting latitudes and longitudes for all the neighborhoods

latitudes = []
longitudes = []
for neighborhood in df['Neighborhood'].tolist():
    g = geocoder.arcgis('{}, Pune, India'.format(str(neighborhood)))
    latitudes.append(g.latlng[0])
    longitudes.append(g.latlng[1])

In [55]:
len(longitudes)

47

In [56]:
#creating a new dataframe with latutude and longitudes

latlng = pd.DataFrame({'Latitude':latitudes, 'Longitude':longitudes})

In [57]:
latlng.head()

,Latitude,Longitude
0,19.00496,73.94583
1,18.56345,73.81227
2,18.54820,73.77318
3,18.51111,73.77776
4,18.51825,73.76570


In [60]:
#concatenating the latitudes and longitudes with the neighborhood df

df = pd.concat([df, latlng], axis=1)

In [61]:
df.head()

,Neighborhood,Latitude,Longitude
0,Ambegaon,19.00496,73.94583
1,Aundh,18.56345,73.81227
2,Baner,18.54820,73.77318
3,Bavdhan Khurd,18.51111,73.77776
4,Bavdhan Budruk,18.51825,73.76570


In [62]:
#saving the dataframe
with open('df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [29]:
#making a copy of dataframe
dfc = df.copy(deep = True)

#### plotting a map of neighborhoods

In [30]:
# create map of Pune using latitude and longitude values
map_pune= folium.Map(location=[18.504220000000032, 73.85302000000007], zoom_start=10)

# add markers to map
for lat, lng,neighborhood in zip(dfc['Latitude'], dfc['Longitude'],dfc['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pune)  
    
map_pune

In [31]:
map_pune.save('map_pune.html')

## Use the Foursquare API to explore the neighborhoods

In [68]:
CLIENT_ID = '****' # your Foursquare ID
CLIENT_SECRET = '****' # your Foursquare Secret
VERSION = '20200314' # Foursquare API version

In [69]:
radius = 2000
LIMIT = 320

venues = []

for lat, long, neighborhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [71]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_df.head()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ambegaon,19.00496,73.94583,Axis Bank ATM,19.000980,73.944656,ATM
1,Ambegaon,19.00496,73.94583,Go cheese world,18.995747,73.944337,Museum
2,Ambegaon,19.00496,73.94583,Axis Bank ATM,19.007980,73.927422,ATM
3,Ambegaon,19.00496,73.94583,Axis Bank ATM,19.007980,73.927422,ATM
4,Aundh,18.56345,73.81227,Westend mall,18.561814,73.807220,Shopping Mall


In [72]:
#no of venues per neighborhood

venues_df.groupby('Neighborhood').count()

,NeighborhoodLatitude,NeighborhoodLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ambegaon,4,4,4,4,4,4
Aundh,56,56,56,56,56,56
Balewadi,53,53,53,53,53,53
Baner,33,33,33,33,33,33
Bavdhan Budruk,26,26,26,26,26,26
Bavdhan Khurd,38,38,38,38,38,38
Bhugaon,10,10,10,10,10,10
Bhukum,4,4,4,4,4,4
Bibvewadi,35,35,35,35,35,35


In [74]:
#saving venues df
with open('venues_df.pkl', 'wb') as f:
    pickle.dump(venues_df, f)

In [6]:
#making venues_df copy
with open('venues_df.pkl', 'rb') as f:
    venues_df = pickle.load(f)
venues_dfc = venues_df.copy(deep = True)

### number of unique venue categories

In [7]:
print('there are {} unique venue categories'.format(len(venues_dfc['VenueCategory'].unique())))

there are 150 unique venue categories


In [79]:
## how many times a venue category occurs
venues_dfc['VenueCategory'].value_counts()

Indian Restaurant                196
Café                             110
Coffee Shop                       69
Fast Food Restaurant              68
Bakery                            47
Restaurant                        46
Snack Place                       45
Pizza Place                       42
Ice Cream Shop                    42
Italian Restaurant                38
Chinese Restaurant                35
Hotel                             33
Lounge                            32
Dessert Shop                      25
Sandwich Place                    24
Vegetarian / Vegan Restaurant     23
Bar                               22
Shopping Mall                     22
Seafood Restaurant                21
Breakfast Spot                    21
Pub                               20
Gym                               19
Asian Restaurant                  18
Diner                             17
Bistro                            16
Donut Shop                        16
Multiplex                         16
D

### Analyze Each Neighborhood

In [8]:
# one hot encoding
venues_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + venues_onehot.columns[:-1].tolist()
venues_onehot = venues_onehot[fixed_columns]

print(venues_onehot.shape)
venues_onehot.head()

(1509, 151)


,Neighborhoods,ATM,Accessories Store,Airport Service,American Restaurant,Arcade,Asian Restaurant,Australian Restaurant,BBQ Joint,Badminton Court,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Yoga Studio,Zoo
0,Ambegaon,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ambegaon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ambegaon,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ambegaon,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aundh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [9]:
venues_grouped = venues_onehot.groupby(["Neighborhoods"]).mean().reset_index()
with open('venues_grouped.pkl', 'wb') as f:
    pickle.dump(venues_grouped, f)
print(venues_grouped.shape)
venues_grouped

(45, 151)


,Neighborhoods,ATM,Accessories Store,Airport Service,American Restaurant,Arcade,Asian Restaurant,Australian Restaurant,BBQ Joint,Badminton Court,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Wine Shop,Yoga Studio,Zoo
0,Ambegaon,0.750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,Aundh,0.000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.017857,0.000,0.000000,0.000000
2,Balewadi,0.000,0.000000,0.000000,0.018868,0.000000,0.018868,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.018868,0.000,0.000000,0.000000
3,Baner,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.030303,0.000,0.000000,0.000000
4,Bavdhan Budruk,0.000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
5,Bavdhan Khurd,0.000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
6,Bhugaon,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
7,Bhukum,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
8,Bibvewadi,0.000,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.028571,0.000,0.000000,0.028571
9,Dhankawadi,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.047619


In [84]:
# how many areas have multiplex

len(venues_grouped[venues_grouped["Multiplex"] > 0])

11

#### Create a new DataFrame for Shopping Mall data only

In [10]:
venues_multiplex = venues_grouped[["Neighborhoods","Multiplex"]]
with open('venues_multiplex.pkl', 'wb') as f:
    pickle.dump(venues_multiplex, f)

In [11]:
venues_multiplex.head()

,Neighborhoods,Multiplex
0,Ambegaon,0.000000
1,Aundh,0.017857
2,Balewadi,0.000000
3,Baner,0.000000
4,Bavdhan Budruk,0.000000


## Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Pune into 3 clusters.

In [12]:
no_of_clusters = 3
venues_muliplex_clustering = venues_multiplex.drop('Neighborhoods', axis=1)
kmeans = KMeans(n_clusters=no_of_clusters, random_state=0).fit(venues_muliplex_clustering)

In [13]:
#creating a new dataframe from venues_multiplex df and adding clustering labels to it

venues_multiplex_clustered = venues_multiplex.copy(deep=True)
venues_multiplex_clustered['ClusterLabels'] = kmeans.labels_
venues_multiplex_clustered.rename(columns = {'Neighborhoods':'Neighborhood'}, inplace=True)
venues_multiplex_clustered.head()

,Neighborhood,Multiplex,ClusterLabels
0,Ambegaon,0.000000,0
1,Aundh,0.017857,2
2,Balewadi,0.000000,0
3,Baner,0.000000,0
4,Bavdhan Budruk,0.000000,0


In [16]:
with open('df.pkl', 'rb') as f:
    df = pickle.load(f)
df.head()

,Neighborhood,Latitude,Longitude
0,Ambegaon,19.00496,73.94583
1,Aundh,18.56345,73.81227
2,Baner,18.54820,73.77318
3,Bavdhan Khurd,18.51111,73.77776
4,Bavdhan Budruk,18.51825,73.76570


In [17]:
#merging df and venues_multiplex_clustered on neighborhood column

venues_multiplex_clustered = venues_multiplex_clustered.merge(df.set_index('Neighborhood'), on='Neighborhood')
venues_multiplex_clustered.head()

,Neighborhood,Multiplex,ClusterLabels,Latitude,Longitude
0,Ambegaon,0.000000,0,19.00496,73.94583
1,Aundh,0.017857,2,18.56345,73.81227
2,Balewadi,0.000000,0,18.57598,73.77983
3,Baner,0.000000,0,18.54820,73.77318
4,Bavdhan Budruk,0.000000,0,18.51825,73.76570


In [18]:
#sorting results by cluster labels
venues_multiplex_clustered.sort_values(['ClusterLabels'], inplace=True)

In [20]:
with open('venues_multiplex_clustered.pkl', 'wb') as f:
    pickle.dump(venues_multiplex_clustered, f)

## Visualizing the clusters

In [21]:
# create map
map_clusters = folium.Map(location=[18.504220000000032, 73.85302000000007], zoom_start=10)

# set color scheme for the clusters
x = np.arange(no_of_clusters)
ys = [i+x+(i*x)**2 for i in range(no_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_multiplex_clustered['Latitude'], venues_multiplex_clustered['Longitude'], venues_multiplex_clustered['Neighborhood'], venues_multiplex_clustered['ClusterLabels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
map_clusters.save('map_clusters.html')

## Cluster 0 

In [25]:
venues_multiplex_clustered[venues_multiplex_clustered['ClusterLabels']==0]

,Neighborhood,Multiplex,ClusterLabels,Latitude,Longitude
0,Ambegaon,0.000000,0,19.004960,73.945830
43,Warje,0.000000,0,18.472110,73.802130
23,Koregaon Park,0.010000,0,18.535330,73.893820
24,Kothrud,0.000000,0,18.505170,73.802450
26,Markal,0.000000,0,18.667570,73.952570
29,Nanded,0.000000,0,18.456420,73.792000
30,Panmala,0.000000,0,18.876470,73.897080
31,Parvati,0.000000,0,18.486970,73.850070
21,Kharadi,0.000000,0,18.544610,73.939250
32,Pashan,0.000000,0,18.536740,73.792900


## Cluster 1

In [26]:
venues_multiplex_clustered[venues_multiplex_clustered['ClusterLabels']==1]

,Neighborhood,Multiplex,ClusterLabels,Latitude,Longitude
37,Undri,0.076923,1,18.4542,73.91719


## Cluster 2

In [27]:
venues_multiplex_clustered[venues_multiplex_clustered['ClusterLabels']==2]

,Neighborhood,Multiplex,ClusterLabels,Latitude,Longitude
28,Mundhwa,0.037037,2,18.530170,73.921250
27,Mohammedwadi,0.043478,2,18.478710,73.915910
34,Shivajinagar,0.020000,2,18.537240,73.838060
25,Manjri,0.023810,2,18.481941,73.865628
14,Ghorpadi,0.023810,2,18.522330,73.897100
15,Hadapsar,0.033898,2,18.502530,73.927060
1,Aundh,0.017857,2,18.563450,73.812270


## Observations

Neighborhoods in cluster 2 have almost all the multiplexes, while those in cluster 0 and 1 have almost none.
Multiplex owners can capitalise on these findings to build multiplexes in these neighborhoods.